In [1]:
!pip install mesa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 9.4 MB/s eta 0:00:00


In [4]:
!pip uninstall -y mesa
!pip install mesa==2.1.1

Found existing installation: Mesa 3.5.0
Uninstalling Mesa-3.5.0:
  Successfully uninstalled Mesa-3.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00


In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

In [3]:
class PersonAgent(Agent):
    def __init__(self, unique_id, model, infection_prob, recovery_prob):
        super().__init__(unique_id, model)
        self.state = "S"  # S = Susceptible, I = Infected, R = Recovered
        self.infection_prob = infection_prob
        self.recovery_prob = recovery_prob

    def step(self):
        if self.state == "I":
            # Try to infect neighbors
            neighbors = self.model.grid.get_neighbors(
                self.pos,
                moore=True,
                include_center=False
            )
            for neighbor in neighbors:
                if neighbor.state == "S" and random.random() < self.infection_prob:
                    neighbor.state = "I"

            # Recovery chance
            if random.random() < self.recovery_prob:
                self.state = "R"

In [4]:
class EpidemicModel(Model):
    def __init__(self, width, height, population,
                 infection_prob, recovery_prob, initial_infected):

        self.num_agents = population
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        self.infection_prob = infection_prob
        self.recovery_prob = recovery_prob

        # Create agents
        for i in range(self.num_agents):
            agent = PersonAgent(i, self, infection_prob, recovery_prob)
            self.schedule.add(agent)

            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

        # Infect initial agents
        infected_agents = random.sample(self.schedule.agents, initial_infected)
        for agent in infected_agents:
            agent.state = "I"

        # Data collector
        self.datacollector = DataCollector(
            model_reporters={
                "Susceptible": lambda m: self.count_state("S"),
                "Infected": lambda m: self.count_state("I"),
                "Recovered": lambda m: self.count_state("R"),
            }
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def count_state(self, state):
        count = 0
        for agent in self.schedule.agents:
            if agent.state == state:
                count += 1
        return count

In [5]:
# Test simulation

model = EpidemicModel(
    width=20,
    height=20,
    population=200,
    infection_prob=0.3,
    recovery_prob=0.1,
    initial_infected=5
)

steps = 50

for i in range(steps):
    model.step()

results = model.datacollector.get_model_vars_dataframe()

results.tail()

,Susceptible,Infected,Recovered
45,41,1,158
46,41,1,158
47,41,1,158
48,41,1,158
49,41,1,158


In [6]:
# Parameter Bounds

BOUNDS = {
    "infection_prob": (0.05, 0.5),
    "recovery_prob": (0.01, 0.3),
    "population": (100, 400),
    "initial_infected": (1, 20),
}

print("Parameter Bounds:")
for k, v in BOUNDS.items():
    print(f"{k}: {v}")

Parameter Bounds:
infection_prob: (0.05, 0.5)
recovery_prob: (0.01, 0.3)
population: (100, 400)
initial_infected: (1, 20)


In [7]:
# Function to run one simulation
def run_simulation():
    infection_prob = random.uniform(*BOUNDS["infection_prob"])
    recovery_prob = random.uniform(*BOUNDS["recovery_prob"])
    population = random.randint(*BOUNDS["population"])
    initial_infected = random.randint(*BOUNDS["initial_infected"])

    model = EpidemicModel(
        width=20,
        height=20,
        population=population,
        infection_prob=infection_prob,
        recovery_prob=recovery_prob,
        initial_infected=initial_infected
    )

    steps = 50
    for _ in range(steps):
        model.step()

    results = model.datacollector.get_model_vars_dataframe()

    peak_infected = results["Infected"].max()
    total_infected = results["Recovered"].iloc[-1]

    return [
        infection_prob,
        recovery_prob,
        population,
        initial_infected,
        peak_infected,
        total_infected
    ]


# Run 1000 simulations
simulation_data = []

for i in range(1000):
    simulation_data.append(run_simulation())

    if (i+1) % 100 == 0:
        print(f"{i+1} simulations completed")

# Create DataFrame
df_sim = pd.DataFrame(simulation_data, columns=[
    "infection_prob",
    "recovery_prob",
    "population",
    "initial_infected",
    "peak_infected",
    "total_infected"
])

df_sim.head()

100 simulations completed
200 simulations completed
300 simulations completed
400 simulations completed
500 simulations completed
600 simulations completed
700 simulations completed
800 simulations completed
900 simulations completed
1000 simulations completed


,infection_prob,recovery_prob,population,initial_infected,peak_infected,total_infected
0,0.111993,0.288728,379,5,51,285
1,0.212946,0.171428,394,3,162,392
2,0.179695,0.208159,171,7,30,64
3,0.119789,0.077628,279,1,12,19
4,0.231591,0.264575,334,3,69,318


In [8]:
from sklearn.model_selection import train_test_split

X = df_sim[[
    "infection_prob",
    "recovery_prob",
    "population",
    "initial_infected"
]]

y = df_sim["peak_infected"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Data split complete")

Data split complete


In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import numpy as np

models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor(),
    "XGBoost": xgb.XGBRegressor(objective="reg:squarederror")
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    r2 = r2_score(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    results.append([name, r2, mae, rmse])

df_results = pd.DataFrame(results, columns=[
    "Model",
    "R2 Score",
    "MAE",
    "RMSE"
])

df_results.sort_values(by="R2 Score", ascending=False)

,Model,R2 Score,MAE,RMSE
4,Gradient Boosting,0.969392,12.093257,15.514371
7,XGBoost,0.967103,12.119113,16.083887
3,Random Forest,0.966120,12.009100,16.322527
0,Linear Regression,0.879887,23.463618,30.733418
1,Ridge,0.871944,24.125787,31.733281
2,Lasso,0.838629,26.481517,35.622814
6,KNN,0.494805,46.246000,63.029647
5,SVR,0.465413,50.051651,64.837266


In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

# Sort by R2 Score
df_results = df_results.sort_values(by="R2 Score", ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(x="Model", y="R2 Score", data=df_results)
plt.xticks(rotation=45)
plt.title("Model Comparison (R2 Score)")
plt.tight_layout()
plt.show()

In [11]:
plt.figure(figsize=(10,5))
sns.barplot(x="Model", y="RMSE", data=df_results)
plt.xticks(rotation=45)
plt.title("Model Comparison (RMSE)")
plt.tight_layout()
plt.show()

In [12]:
best_model = df_results.iloc[0]
print("Best Model:")
print(best_model)

Best Model:
Model       Gradient Boosting
R2 Score             0.969392
MAE                 12.093257
RMSE                15.514371
Name: 4, dtype: object
